> 📌 **Avant de commencer :**
>
> [![Ouvrir dans Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/11kJfWc-jEEZusoM6vEE0WKxkGoaXEtU2?usp=sharing)
>
> 🔁 *Pensez à faire une copie dans votre Google Drive (`Fichier > Enregistrer une copie dans Drive`) pour pouvoir l’éditer librement.*


# Traduction Automatique Neuronale

Bienvenue dans votre premier devoir de programmation de la semaine !

* Vous allez construire un modèle de Traduction Automatique Neuronale (NMT) pour traduire des dates lisibles par l’humain ("25th of June, 2009") en dates lisibles par une machine ("2009-06-25").
* Vous réaliserez cela en utilisant un modèle d’attention, l’un des modèles séquence-à-séquence les plus sophistiqués.



## Table des matières

- [Packages](#0)
- [1 - Traduction des dates lisibles par l'humain en dates lisibles par la machine](#1)
    - [1.1 - Jeu de données](#1-1)
- [2 - Traduction automatique neuronale avec attention](#2)
    - [2.1 - Mécanisme d'attention](#2-1)
        - [Exercice 1 - one_step_attention](#ex-1)
        - [Exercice 2 - modelf](#ex-2)
        - [Exercice 3 - Compiler le modèle](#ex-3)
- [3 - Visualisation de l'attention (Optionnel / Non noté)](#3)
    - [3.1 - Obtenir les poids d'attention du réseau](#3-1)


## Installation des paquets

In [ ]:
! pip install faker babel tqdm tensorflow transformers datasets evaluate scikit-learn matplotlib
! pip install solutions
! pip install testCase

<a name='0'></a>
## Packages

In [ ]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
#from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline


from termcolor import colored

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RepeatVector

In [ ]:
def generateTestCases():
	testCases = {
	    'one_step_attention': {
	        'partId': 'zcQIs',
	        'testCases': [
	            {
	                'testInput': 0,
	                'testOutput': m_out2
	            }
	        ]
	    },
	    'model': {
	        'partId': 'PTKef',
	        'testCases': [
	            {
	                'testInput': (Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size),
	                'testOutput': m_out1
	            }
	        ]
	    }
       }
	return testCases

In [ ]:
fake = Faker()
Faker.seed(12345)
random.seed(12345)

# Définir le format des données que nous souhaitons générer
FORMATS = ['short',
           'medium',
           'long',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'full',
           'd MMM YYY',
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']

# changer cela si vous voulez utiliser une autre langue
LOCALES = ['en_US']

def load_date():
    """
        Charge quelques dates factices
        :returns: tuple contenant la chaîne lisible par l'humain, la chaîne lisible par la machine, et l'objet date
    """
    dt = fake.date_object()

    try:
        human_readable = format_date(dt, format=random.choice(FORMATS),  locale='en_US')  # locale=random.choice(LOCALES))
        human_readable = human_readable.lower()
        human_readable = human_readable.replace(',','')
        machine_readable = dt.isoformat()

    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

def load_dataset(m):
    """
        Charge un jeu de données avec m exemples et vocabulaires
        :m: le nombre d'exemples à générer
    """

    human_vocab = set()
    machine_vocab = set()
    dataset = []
    Tx = 30

    for i in tqdm(range(m)):
        h, m, _ = load_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))

    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'],
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v:k for k,v in inv_machine.items()}

    return dataset, human, machine, inv_machine

def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    X, Y = zip(*dataset)

    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]

    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

def string_to_int(string, length, vocab):
    """
    Convertit toutes les chaînes du vocabulaire en liste d'entiers représentant les positions
    des caractères de la chaîne d'entrée dans le vocabulaire "vocab".

    Arguments:
    string -- chaîne d'entrée, par ex. 'Wed 10 Jul 2007'
    length -- nombre de pas de temps souhaités, détermine si la sortie sera remplie ou tronquée
    vocab -- vocabulaire, dictionnaire utilisé pour indexer chaque caractère de la chaîne

    Returns:
    rep -- liste d'entiers (ou '<unk>') (taille = length) représentant la position des caractères dans le vocabulaire
    """

    # mise en minuscules pour standardiser
    string = string.lower()
    string = string.replace(',','')

    if len(string) > length:
        string = string[:length]

    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))

    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))

    return rep

def int_to_string(ints, inv_vocab):
    """
    Produit une liste de caractères lisibles par machine à partir d'une liste d'indices dans le vocabulaire machine.

    Arguments:
    ints -- liste d'entiers représentant les indices dans le vocabulaire machine
    inv_vocab -- dictionnaire mappant les indices aux caractères lisibles par machine

    Returns:
    l -- liste des caractères correspondant aux indices grâce au mapping inv_vocab
    """

    l = [inv_vocab[i] for i in ints]
    return l


EXAMPLES = ['3 May 1979', '5 Apr 09', '20th February 2016', 'Wed 10 Jul 2007']

def run_example(model, input_vocabulary, inv_output_vocabulary, text):
    encoded = string_to_int(text, TIME_STEPS, input_vocabulary)
    prediction = model.predict(np.array([encoded]))
    prediction = np.argmax(prediction[0], axis=-1)
    return int_to_string(prediction, inv_output_vocabulary)

def run_examples(model, input_vocabulary, inv_output_vocabulary, examples=EXAMPLES):
    predicted = []
    for example in examples:
        predicted.append(''.join(run_example(model, input_vocabulary, inv_output_vocabulary, example)))
        print('entrée :', example)
        print('sortie :', predicted[-1])
    return predicted


def softmax(x, axis=1):
    """Fonction d'activation softmax.
    # Arguments
        x : tenseur.
        axis: entier, axe selon lequel la normalisation softmax est appliquée.
    # Returns
        tenseur, sortie de la transformation softmax.
    # Raises
        ValueError: en cas de tenseur 1D.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Impossible d\'appliquer softmax sur un tenseur 1D')


def plot_attention_map(modelx, input_vocabulary, inv_output_vocabulary, text, n_s = 128, num = 7):
    """
    Trace la carte d'attention.
    """
    attention_map = np.zeros((10, 30))
    layer = modelx.get_layer('attention_weights')

    Ty, Tx = attention_map.shape

    human_vocab_size = 37

    # C'est un peu compliqué mais cette version de tensorflow-keras contient un bug
    # qui affecte la réutilisation des couches dans un modèle avec l'API fonctionnelle.
    # Donc je dois recréer le modèle en fonction des composants fonctionnels
    # et les connecter un par un.
    # idéalement cela peut se faire simplement ainsi :
    # layer = modelx.layers[num]
    # f = Model(modelx.inputs, [layer.get_output_at(t) for t in range(Ty)])
    #

    X = modelx.inputs[0]
    s0 = modelx.inputs[1]
    c0 = modelx.inputs[2]
    s = s0
    c = s0

    a = modelx.layers[2](X)
    outputs = []

    for t in range(Ty):
        s_prev = s
        s_prev = modelx.layers[3](s_prev)
        concat = modelx.layers[4]([a, s_prev])
        e = modelx.layers[5](concat)
        energies = modelx.layers[6](e)
        alphas = modelx.layers[7](energies)
        context = modelx.layers[8]([alphas, a])
        # N'oubliez pas de passer : initial_state = [état caché, état cellule] (≈ 1 ligne)
        s, _, c = modelx.layers[10](context, initial_state = [s, c])
        outputs.append(energies)

    f = Model(inputs=[X, s0, c0], outputs=outputs)

    s0 = np.zeros((1, n_s))
    c0 = np.zeros((1, n_s))
    encoded = np.array(string_to_int(text, Tx, input_vocabulary)).reshape((1, 30))
    encoded = np.array(list(map(lambda x: to_categorical(x, num_classes=len(input_vocabulary)), encoded)))

    r = f([encoded, s0, c0])

    for t in range(Ty):
        for t_prime in range(Tx):
            attention_map[t][t_prime] = r[t][0, t_prime]

    # Normaliser la carte d'attention
    row_max = attention_map.max(axis=1)
    attention_map = attention_map / row_max[:, None]

    prediction = modelx.predict([encoded, s0, c0])

    predicted_text = []
    for i in range(len(prediction)):
        predicted_text.append(int(np.argmax(prediction[i], axis=1)))

    predicted_text = list(predicted_text)
    predicted_text = int_to_string(predicted_text, inv_output_vocabulary)
    text_ = list(text)

    # obtenir les longueurs des chaînes
    input_length = len(text)
    output_length = Ty

    # Tracer la carte d'attention
    plt.clf()
    f = plt.figure(figsize=(8, 8.5))
    ax = f.add_subplot(1, 1, 1)

    # ajouter l'image
    i = ax.imshow(attention_map, interpolation='nearest', cmap='Blues')

    # ajouter la barre de couleur
    cbaxes = f.add_axes([0.2, 0, 0.6, 0.03])
    cbar = f.colorbar(i, cax=cbaxes, orientation='horizontal')
    cbar.ax.set_xlabel('Valeur Alpha (probabilité de sortie du "softmax")', labelpad=2)

    # ajouter les labels
    ax.set_yticks(range(output_length))
    ax.set_yticklabels(predicted_text[:output_length])

    ax.set_xticks(range(input_length))
    ax.set_xticklabels(text_[:input_length], rotation=45)

    ax.set_xlabel('Séquence d\'entrée')
    ax.set_ylabel('Séquence de sortie')

    # ajouter la grille et la légende
    ax.grid()

    #f.show()

    return attention_map


In [ ]:
# Compare les deux entrées
def comparator(learner, instructor):
    if len(learner) != len(instructor):
        raise AssertionError("Erreur dans le test. Les listes contiennent un nombre différent d'éléments")
    for index, a in enumerate(instructor):
        b = learner[index]
        if tuple(a) != tuple(b):
            print(colored(f"Test échoué à l'index {index}", attrs=['bold']),
                  "\n Valeur attendue \n\n", colored(f"{a}", "green"),
                  "\n\n ne correspond pas à la valeur entrée : \n\n",
                  colored(f"{b}", "red"))
            raise AssertionError("Erreur dans le test")
    print(colored("Tous les tests sont passés avec succès !", "green"))

# Extrait la description d'un modèle donné
def summary(model):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    result = []
    for layer in model.layers:
        descriptors = [layer.__class__.__name__, layer.output_shape, layer.count_params()]
        if (type(layer) == Conv2D):
            descriptors.append(layer.padding)
            descriptors.append(layer.activation.__name__)
            descriptors.append(layer.kernel_initializer.__class__.__name__)
        if (type(layer) == MaxPooling2D):
            descriptors.append(layer.pool_size)
            descriptors.append(layer.strides)
            descriptors.append(layer.padding)
        if (type(layer) == Dropout):
            descriptors.append(layer.rate)
        if (type(layer) == ZeroPadding2D):
            descriptors.append(layer.padding)
        if (type(layer) == Dense):
            descriptors.append(layer.activation.__name__)
        if (type(layer) == LSTM):
            descriptors.append(layer.input_shape)
            descriptors.append(layer.activation.__name__)
        if (type(layer) == RepeatVector):
            descriptors.append(layer.n)
        result.append(descriptors)
    return result


<a name='1'></a>  
## 1 - Traduction des dates lisibles par l’humain en dates lisibles par la machine

* Le modèle que vous allez construire ici pourrait être utilisé pour traduire d’une langue à une autre, par exemple de l’anglais vers l’hindi.  
* Cependant, la traduction automatique nécessite des jeux de données massifs et prend généralement plusieurs jours d’entraînement sur GPU.  
* Pour vous offrir un terrain d’expérimentation avec ces modèles sans utiliser de très grands jeux de données, nous allons effectuer une tâche plus simple de « traduction de dates ».  
* Le réseau prendra en entrée une date écrite dans différents formats possibles (*par exemple : "le 29 août 1958", "30/03/1968", "24 JUIN 1987"*).  
* Le réseau traduira ces formats en dates standardisées et lisibles par une machine (*par exemple : "1958-08-29", "1968-03-30", "1987-06-24"*).  
* Nous apprendrons au réseau à produire les dates dans le format commun lisible par machine : AAAA-MM-JJ.

<!--
Prenez un moment pour jeter un œil à [nmt_utils.py](./nmt_utils.py) pour voir tous les formats. Comptez et comprenez comment les formats fonctionnent, vous aurez besoin de cette connaissance plus tard. !-->


<a name='1-1'></a>  
### 1.1 - Jeu de données

Nous allons entraîner le modèle sur un jeu de données contenant 10 000 dates lisibles par l’humain ainsi que leurs équivalents standardisés et lisibles par la machine.  
Exécutons les cellules suivantes pour charger le jeu de données et afficher quelques exemples.


In [ ]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

In [ ]:
dataset[:10]

Vous avez chargé :  
- `dataset` : une liste de tuples (date lisible par l’humain, date lisible par la machine).  
- `human_vocab` : un dictionnaire Python mappant tous les caractères utilisés dans les dates lisibles par l’humain à un indice entier.  
- `machine_vocab` : un dictionnaire Python mappant tous les caractères utilisés dans les dates lisibles par la machine à un indice entier.  
    - **Remarque** : ces indices ne sont pas forcément les mêmes que ceux de `human_vocab`.  
- `inv_machine_vocab` : le dictionnaire inverse de `machine_vocab`, qui mappe les indices vers les caractères.

Prétraitons maintenant les données pour convertir les textes bruts en indices.  
- Nous fixons Tx = 30  
    - Tx correspond à la longueur maximale d’une date lisible par l’humain.  
    - Si une entrée est plus longue, elle sera tronquée.  
- Nous fixons Ty = 10  
    - Le format "AAAA-MM-JJ" fait 10 caractères.


In [ ]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

Vous avez maintenant :  
- `X` : une version prétraitée des dates lisibles par l’humain dans le jeu d’entraînement.  
    - Chaque caractère dans `X` est remplacé par un indice (entier) mappé au caractère via `human_vocab`.  
    - Chaque date est complétée par un padding pour assurer une longueur de $T_x$, à l’aide d’un caractère spécial (<pad>).  
    - `X.shape = (m, Tx)` où `m` est le nombre d’exemples dans le batch d’entraînement.  

- `Y` : une version prétraitée des dates lisibles par la machine dans le jeu d’entraînement.  
    - Chaque caractère est remplacé par l’indice correspondant dans `machine_vocab`.  
    - `Y.shape = (m, Ty)`.  

- `Xoh` : version one-hot de `X`  
    - Chaque indice dans `X` est converti en représentation one-hot (si l’indice est 2, la position 2 dans le vecteur one-hot vaut 1, les autres positions valent 0).  
    - `Xoh.shape = (m, Tx, len(human_vocab))`.  

- `Yoh` : version one-hot de `Y`  
    - Chaque indice dans `Y` est converti en représentation one-hot.  
    - `Yoh.shape = (m, Ty, len(machine_vocab))`.  
    - `len(machine_vocab) = 11` car il y a 10 chiffres (0 à 9) plus le symbole `-`.


* Regardons également quelques exemples d’exemples d’entraînement prétraités.  
* N’hésitez pas à modifier la valeur de `index` dans la cellule ci-dessous pour parcourir le jeu de données et voir comment les dates source/cible sont prétraitées.


In [ ]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

<a name='2'></a>  
## 2 - Traduction automatique neuronale avec attention

* Si vous deviez traduire un paragraphe d’un livre du français vers l’anglais, vous ne liriez pas tout le paragraphe, puis fermeriez le livre avant de traduire.  
* Même pendant la traduction, vous liriez et reliriez certaines parties, en vous concentrant sur les passages français correspondant à ceux de l’anglais que vous écrivez.  
* Le mécanisme d’attention indique à un modèle de traduction automatique neuronale où il doit porter son attention à chaque étape.

<a name='2-1'></a>  
### 2.1 - Mécanisme d’attention

Dans cette partie, vous allez implémenter le mécanisme d’attention présenté dans les vidéos de cours.  
* Voici une figure pour vous rappeler le fonctionnement du modèle.  
  * Le schéma de gauche montre le modèle d’attention.  
  * Le schéma de droite montre ce que fait une étape d’« attention » pour calculer les variables d’attention $\alpha^{\langle t, t' \rangle}$.  
  * Ces variables $\alpha^{\langle t, t' \rangle}$ sont utilisées pour calculer la variable contexte $context^{\langle t \rangle}$ à chaque pas de temps en sortie ($t=1, \ldots, T_y$).


<table>
<td>
<img src="https://images2018.cnblogs.com/blog/937910/201804/937910-20180408225129369-1491566692.png" style="width:500;height:500px;"> <br>
</td>
<td>
<img src="https://images2018.cnblogs.com/blog/937910/201804/937910-20180408232000910-186858891.png" style="width:500;height:500px;"> <br>
</td>
</table>
<caption><center> **Figure 1** : Traduction automatique neuronale avec attention</center></caption>


Voici quelques propriétés du modèle que vous pouvez remarquer :

#### LSTMs avant et après l’attention de chaque côté du mécanisme d’attention

- Il y a deux LSTMs distincts dans ce modèle (voir le diagramme à gauche) : les LSTMs *avant* l’attention et *après* l’attention.  
- La Bi-LSTM *avant l’attention* est celle en bas de l’image, c’est une LSTM bidirectionnelle et elle se situe *avant* le mécanisme d’attention.  
    - Le mécanisme d’attention est montré au centre du diagramme de gauche.  
    - La Bi-LSTM avant attention s’étend sur $T_x$ pas de temps.  
- La LSTM *après l’attention* est en haut du diagramme et se situe *après* le mécanisme d’attention.  
    - La LSTM après attention s’étend sur $T_y$ pas de temps.  

- La LSTM après attention transmet l’état caché $s^{\langle t \rangle}$ et l’état de la cellule $c^{\langle t \rangle}$ d’un pas de temps au suivant.


#### An LSTM has both a hidden state and cell state
* In the lecture videos, we were using only a basic RNN for the post-attention sequence model
    * This means that the state captured by the RNN was outputting only the hidden state $s^{\langle t\rangle}$.
* In this assignment, we are using an LSTM instead of a basic RNN.
    * So the LSTM has both the hidden state $s^{\langle t\rangle}$ and the cell state $c^{\langle t\rangle}$.

#### Each time step does not use predictions from the previous time step
* Unlike previous text generation examples earlier in the course, in this model, the post-attention LSTM at time $t$ does not take the previous time step's prediction $y^{\langle t-1 \rangle}$ as input.
* The post-attention LSTM at time 't' only takes the hidden state $s^{\langle t\rangle}$ and cell state $c^{\langle t\rangle}$ as input.
* We have designed the model this way because unlike language generation (where adjacent characters are highly correlated) there isn't as strong a dependency between the previous character and the next character in a YYYY-MM-DD date.

#### Concatenation of hidden states from the forward and backward pre-attention LSTMs
- $\overrightarrow{a}^{\langle t \rangle}$: hidden state of the forward-direction, pre-attention LSTM.
- $\overleftarrow{a}^{\langle t \rangle}$: hidden state of the backward-direction, pre-attention LSTM.
- $a^{\langle t \rangle} = [\overrightarrow{a}^{\langle t \rangle}, \overleftarrow{a}^{\langle t \rangle}]$: the concatenation of the activations of both the forward-direction $\overrightarrow{a}^{\langle t \rangle}$ and backward-directions $\overleftarrow{a}^{\langle t \rangle}$ of the pre-attention Bi-LSTM.

#### Computing "energies" $e^{\langle t, t' \rangle}$ as a function of $s^{\langle t-1 \rangle}$ and $a^{\langle t' \rangle}$
- Recall in the lesson videos "Attention Model", at time 6:45 to 8:16, the definition of "e" as a function of $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$.
    - "e" is called the "energies" variable.
    - $s^{\langle t-1 \rangle}$ is the hidden state of the post-attention LSTM
    - $a^{\langle t' \rangle}$ is the hidden state of the pre-attention LSTM.
    - $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ are fed into a simple neural network, which learns the function to output $e^{\langle t, t' \rangle}$.
    - $e^{\langle t, t' \rangle}$ is then used when computing the attention $\alpha^{\langle t, t' \rangle}$ that $y^{\langle t \rangle}$ should pay to $a^{\langle t' \rangle}$.

- The diagram on the right of figure 1 uses a `RepeatVector` node to copy $s^{\langle t-1 \rangle}$'s value $T_x$ times.
- Then it uses `Concatenation` to concatenate $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$.
- The concatenation of $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ is fed into a "Dense" layer, which computes $e^{\langle t, t' \rangle}$.
- $e^{\langle t, t' \rangle}$ is then passed through a softmax to compute $\alpha^{\langle t, t' \rangle}$.
- Note that the diagram doesn't explicitly show variable $e^{\langle t, t' \rangle}$, but $e^{\langle t, t' \rangle}$ is above the Dense layer and below the Softmax layer in the diagram in the right half of figure 1.
- We'll explain how to use `RepeatVector` and `Concatenation` in Keras below.

#### Implementation Details
   
Let's implement this neural translator. You will start by implementing two functions: `one_step_attention()` and `model()`.

#### one_step_attention
* The inputs to the one_step_attention at time step $t$ are:
    - $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$: all hidden states of the pre-attention Bi-LSTM.
    - $s^{<t-1>}$: the previous hidden state of the post-attention LSTM
* one_step_attention computes:
    - $[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$: the attention weights
    - $context^{ \langle t \rangle }$: the context vector:
    
$$context^{<t>} = \sum_{t' = 1}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$

##### Clarifying 'context' and 'c'
- In the lecture videos, the context was denoted $c^{\langle t \rangle}$
- In the assignment, we are calling the context $context^{\langle t \rangle}$.
    - This is to avoid confusion with the post-attention LSTM's internal memory cell variable, which is also denoted $c^{\langle t \rangle}$.

<a name='ex-1'></a>
### Exercise 1 - one_step_attention

Implement `one_step_attention()`.

* The function `model()` will call the layers in `one_step_attention()` $T_y$ times using a for-loop.
* It is important that all $T_y$ copies have the same weights.
    * It should not reinitialize the weights every time.
    * In other words, all $T_y$ steps should have shared weights.
* Here's how you can implement layers with shareable weights in Keras:
    1. Define the layer objects in a variable scope that is outside of the `one_step_attention` function.  For example, defining the objects as global variables would work.
        - Note that defining these variables inside the scope of the function `model` would technically work, since `model` will then call the `one_step_attention` function.  For the purposes of making grading and troubleshooting easier, we are defining these as global variables.  Note that the automatic grader will expect these to be global variables as well.
    2. Call these objects when propagating the input.
* We have defined the layers you need as global variables.
    * Please run the following cells to create them.
    * Please note that the automatic grader expects these global variables with the given variable names.  For grading purposes, please do not rename the global variables.
* Please check the Keras documentation to learn more about these layers.  The layers are functions.  Below are examples of how to call these functions.
    * [RepeatVector()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RepeatVector)
```Python
var_repeated = repeat_layer(var1)
```
    * [Concatenate()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Concatenate)   
```Python
concatenated_vars = concatenate_layer([var1,var2,var3])
```
    * [Dense()](https://keras.io/layers/core/#dense)  
```Python
var_out = dense_layer(var_in)
```
    * [Activation()](https://keras.io/layers/core/#activation)  
```Python
activation = activation_layer(var_in)  
```
    * [Dot()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dot)  
```Python
dot_product = dot_layer([var1,var2])
```

In [ ]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [ ]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: one_step_attention

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.

    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)

    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """

    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev =
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat =
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e =
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies =
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas =
    # Use dotor together with "alphas" and "a", in this order, to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context =
    ### END CODE HERE ###

    return context

In [ ]:
# UNIT TEST
def one_step_attention_test(target):

    m = 10
    Tx = 30
    n_a = 32
    n_s = 64
    #np.random.seed(10)
    a = np.random.uniform(1, 0, (m, Tx, 2 * n_a)).astype(np.float32)
    s_prev =np.random.uniform(1, 0, (m, n_s)).astype(np.float32) * 1
    context = target(a, s_prev)

    assert type(context) == tf.python.framework.ops.EagerTensor, "Unexpected type. It should be a Tensor"
    assert tuple(context.shape) == (m, 1, n_s), "Unexpected output shape"
    assert np.all(context.numpy() > 0), "All output values must be > 0 in this example"
    assert np.all(context.numpy() < 1), "All output values must be < 1 in this example"

    #assert np.allclose(context[0][0][0:5].numpy(), [0.50877404, 0.57160693, 0.45448175, 0.50074816, 0.53651875]), "Unexpected values in the result"
    print("\033[92mAll tests passed!")

one_step_attention_test(one_step_attention)

<a name='ex-2'></a>
### Exercise 2 - modelf

Implement `modelf()` as explained in figure 1 and the instructions:

* `modelf` first runs the input through a Bi-LSTM to get $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$.
* Then, `modelf` calls `one_step_attention()` $T_y$ times using a `for` loop.  At each iteration of this loop:
    - It gives the computed context vector $context^{<t>}$ to the post-attention LSTM.
    - It runs the output of the post-attention LSTM through a dense layer with softmax activation.
    - The softmax generates a prediction $\hat{y}^{<t>}$.
    
Again, we have defined global layers that will share weights to be used in `modelf()`.

In [ ]:
n_a = 32 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 64 # number of units for the post-attention LSTM's hidden state "s"

# Please note, this is the post attention LSTM cell.
post_activation_LSTM_cell = LSTM(n_s, return_state = True) # Please do not modify this global variable.
output_layer = Dense(len(machine_vocab), activation=softmax)

Now you can use these layers $T_y$ times in a `for` loop to generate the outputs, and their parameters will not be reinitialized. You will have to carry out the following steps:

1. Propagate the input `X` into a bi-directional LSTM.
    * [Bidirectional](https://keras.io/layers/wrappers/#bidirectional)
    * [LSTM](https://keras.io/layers/recurrent/#lstm)
    * Remember that we want the LSTM to return a full sequence instead of just the last hidden state.  
    
Sample code:

```Python
sequence_of_hidden_states = Bidirectional(LSTM(units=..., return_sequences=...))(the_input_X)
```
    
2. Iterate for $t = 0, \cdots, T_y-1$:
    1. Call `one_step_attention()`, passing in the sequence of hidden states $[a^{\langle 1 \rangle},a^{\langle 2 \rangle}, ..., a^{ \langle T_x \rangle}]$ from the pre-attention bi-directional LSTM, and the previous hidden state $s^{<t-1>}$ from the post-attention LSTM to calculate the context vector $context^{<t>}$.
    2. Give $context^{<t>}$ to the post-attention LSTM cell.
        - Remember to pass in the previous hidden-state $s^{\langle t-1\rangle}$ and cell-states $c^{\langle t-1\rangle}$ of this LSTM
        * This outputs the new hidden state $s^{<t>}$ and the new cell state $c^{<t>}$.  

        Sample code:
        ```Python
        next_hidden_state, _ , next_cell_state =
            post_activation_LSTM_cell(inputs=..., initial_state=[prev_hidden_state, prev_cell_state])
        ```   
        Please note that the layer is actually the "post attention LSTM cell".  For the purposes of passing the automatic grader, please do not modify the naming of this global variable.  This will be fixed when we deploy updates to the automatic grader.
    3. Apply a dense, softmax layer to $s^{<t>}$, get the output.  
        Sample code:
        ```Python
        output = output_layer(inputs=...)
        ```
    4. Save the output by adding it to the list of outputs.

3. Create your Keras model instance.
    * It should have three inputs:
        * `X`, the one-hot encoded inputs to the model, of shape ($T_{x}, humanVocabSize)$
        * $s^{\langle 0 \rangle}$, the initial hidden state of the post-attention LSTM
        * $c^{\langle 0 \rangle}$, the initial cell state of the post-attention LSTM
    * The output is the list of outputs.  
    Sample code
    ```Python
    model = Model(inputs=[...,...,...], outputs=...)
    ```

In [ ]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: model

def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """

    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0

    # Initialize empty list of outputs
    outputs = []

    ### START CODE HERE ###

    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    a =

    # Step 2: Iterate for Ty steps
    for t in range(Ty):

        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context =

        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c =

        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out =

        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)


    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model =

    ### END CODE HERE ###

    return model

In [ ]:
# UNIT TEST
from test_utils import *

def modelf_test(target):
    m = 10
    Tx = 30
    n_a = 32
    n_s = 64
    len_human_vocab = 37
    len_machine_vocab = 11


    model = target(Tx, Ty, n_a, n_s, len_human_vocab, len_machine_vocab)

    print(summary(model))


    expected_summary = [['InputLayer', [(None, 30, 37)], 0],
                         ['InputLayer', [(None, 64)], 0],
                         ['Bidirectional', (None, 30, 64), 17920],
                         ['RepeatVector', (None, 30, 64), 0, 30],
                         ['Concatenate', (None, 30, 128), 0],
                         ['Dense', (None, 30, 10), 1290, 'tanh'],
                         ['Dense', (None, 30, 1), 11, 'relu'],
                         ['Activation', (None, 30, 1), 0],
                         ['Dot', (None, 1, 64), 0],
                         ['InputLayer', [(None, 64)], 0],
                         ['LSTM',[(None, 64), (None, 64), (None, 64)], 33024,[(None, 1, 64), (None, 64), (None, 64)],'tanh'],
                         ['Dense', (None, 11), 715, 'softmax']]

    assert len(model.outputs) == 10, f"Wrong output shape. Expected 10 != {len(model.outputs)}"

    comparator(summary(model), expected_summary)


modelf_test(modelf)

Run the following cell to create your model.

In [ ]:
model = modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

#### Troubleshooting Note
* If you are getting repeated errors after an initially incorrect implementation of "model", but believe that you have corrected the error, you may still see error messages when building your model.  
* A solution is to save and restart your kernel (or shutdown then restart your notebook), and re-run the cells.

Let's get a summary of the model to check if it matches the expected output.

In [ ]:
model.summary()

**Expected Output**:

Here is the summary you should see
<table>
    <tr>
        <td>
            **Total params:**
        </td>
        <td>
         52,960
        </td>
    </tr>
        <tr>
        <td>
            **Trainable params:**
        </td>
        <td>
         52,960
        </td>
    </tr>
            <tr>
        <td>
            **Non-trainable params:**
        </td>
        <td>
         0
        </td>
    </tr>
                    <tr>
        <td>
            **bidirectional_1's output shape **
        </td>
        <td>
         (None, 30, 64)  
        </td>
    </tr>
    <tr>
        <td>
            **repeat_vector_1's output shape **
        </td>
        <td>
         (None, 30, 64)
        </td>
    </tr>
                <tr>
        <td>
            **concatenate_1's output shape **
        </td>
        <td>
         (None, 30, 128)
        </td>
    </tr>
            <tr>
        <td>
            **attention_weights's output shape **
        </td>
        <td>
         (None, 30, 1)  
        </td>
    </tr>
        <tr>
        <td>
            **dot_1's output shape **
        </td>
        <td>
         (None, 1, 64)
        </td>
    </tr>
           <tr>
        <td>
            **dense_3's output shape **
        </td>
        <td>
         (None, 11)
        </td>
    </tr>
</table>


<a name='ex-3'></a>
### Exercise 3 - Compile the Model

* After creating your model in Keras, you need to compile it and define the loss function, optimizer and metrics you want to use.
    * Loss function: 'categorical_crossentropy'.
    * Optimizer: [Adam](https://keras.io/optimizers/#adam) [optimizer](https://keras.io/optimizers/#usage-of-optimizers)
        - learning rate = 0.005
        - $\beta_1 = 0.9$
        - $\beta_2 = 0.999$
        - decay = 0.01  
    * metric: 'accuracy'
    
Sample code
```Python
optimizer = Adam(lr=..., beta_1=..., beta_2=..., decay=...)
model.compile(optimizer=..., loss=..., metrics=[...])
```

In [ ]:
### START CODE HERE ### (≈2 lines)


### END CODE HERE ###

In [ ]:
# UNIT TESTS
assert opt.lr == 0.005, "Set the lr parameter to 0.005"
assert opt.beta_1 == 0.9, "Set the beta_1 parameter to 0.9"
assert opt.beta_2 == 0.999, "Set the beta_2 parameter to 0.999"
assert opt.decay == 0.01, "Set the decay parameter to 0.01"
assert model.loss == "categorical_crossentropy", "Wrong loss. Use 'categorical_crossentropy'"
assert model.optimizer == opt, "Use the optimizer that you have instantiated"
assert model.compiled_metrics._user_metrics[0] == 'accuracy', "set metrics to ['accuracy']"

print("\033[92mAll tests passed!")

#### Define inputs and outputs, and fit the model
The last step is to define all your inputs and outputs to fit the model:
- You have input `Xoh` of shape $(m = 10000, T_x = 30, human\_vocab=37)$ containing the training examples.
- You need to create `s0` and `c0` to initialize your `post_attention_LSTM_cell` with zeros.
- Given the `model()` you coded, you need the "outputs" to be a list of 10 elements of shape (m, T_y).
    - The list `outputs[i][0], ..., outputs[i][Ty]` represents the true labels (characters) corresponding to the $i^{th}$ training example (`Xoh[i]`).
    - `outputs[i][j]` is the true label of the $j^{th}$ character in the $i^{th}$ training example.

In [ ]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

Let's now fit the model and run it for one epoch.

In [ ]:
model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100)

Pendant l'entraînement, vous pouvez voir la perte ainsi que la précision pour chacune des 10 positions de la sortie. Le tableau ci-dessous donne un exemple de ce que les précisions pourraient être si le batch contenait 2 exemples :

<img src="https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQFp5Dn_qd6LpYqVJJhAgfGHh88O8ATGZylV-61NRP6OW88kTqj" style="width:700;height:200px;"> <br>
<caption><center>Par exemple, `dense_2_acc_8: 0.89` signifie que vous prédisez correctement le 7ème caractère de la sortie 89% du temps sur le batch de données courant.</center></caption>

Nous avons entraîné ce modèle plus longtemps et sauvegardé les poids. Exécutez la cellule suivante pour charger nos poids. (En entraînant un modèle plusieurs minutes, vous devriez obtenir une précision similaire, mais charger notre modèle vous fera gagner du temps.)


In [ ]:
model.load_weights('models/model.h5')

You can now see the results on new examples.

In [ ]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
s00 = np.zeros((1, n_s))
c00 = np.zeros((1, n_s))
for example in EXAMPLES:
    source = string_to_int(example, Tx, human_vocab)
    #print(source)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source = np.swapaxes(source, 0, 1)
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s00, c00])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    print("source:", example)
    print("output:", ''.join(output),"\n")

You can also change these examples to test with your own examples. The next part will give you a better sense of what the attention mechanism is doing--i.e., what part of the input the network is paying attention to when generating a particular output character.

<a name='3'></a>
## 3 - Visualisation de l'Attention (Optionnel / Non noté)

Puisque le problème a une longueur de sortie fixe de 10, il est aussi possible de réaliser cette tâche en utilisant 10 unités softmax différentes pour générer les 10 caractères de la sortie. Mais un avantage du modèle d’attention est que chaque partie de la sortie (comme le mois) sait qu’elle doit dépendre seulement d’une petite partie de l’entrée (les caractères dans l’entrée donnant le mois). On peut visualiser quelle partie de l’entrée chaque partie de la sortie regarde.

Considérons la tâche de traduire « Saturday 9 May 2018 » en « 2018-05-09 ». Si on visualise les $\alpha^{\langle t, t' \rangle}$ calculés, on obtient ceci :

<img src="https://wiki.hacksmeta.com/static/images/ML/DL/attention/date_attention.png" style="width:600;height:300px;"> <br>
<caption><center> **Figure 8** : Carte complète d’attention</center></caption>

Remarquez comment la sortie ignore la partie « Saturday » de l’entrée. Aucun des pas de temps en sortie ne prête beaucoup d’attention à cette partie de l’entrée. On voit aussi que le 9 a été traduit en 09 et que May a été correctement traduit en 05, la sortie prêtant attention aux parties de l’entrée nécessaires pour faire la traduction. L’année nécessite surtout que le modèle prête attention au « 18 » dans l’entrée afin de générer « 2018 ».


<a name='3-1'></a>
### 3.1 - Getting the Attention Weights From the Network

Lets now visualize the attention values in your network. We'll propagate an example through the network, then visualize the values of $\alpha^{\langle t, t' \rangle}$.

To figure out where the attention values are located, let's start by printing a summary of the model .

In [ ]:
model.summary()

Navigate through the output of `model.summary()` above. You can see that the layer named `attention_weights` outputs the `alphas` of shape (m, 30, 1) before `dot_2` computes the context vector for every time step $t = 0, \ldots, T_y-1$. Let's get the attention weights from this layer.

The function `attention_map()` pulls out the attention values from your model and plots them.

**Note**: We are aware that you might run into an error running the cell below despite a valid implementation for Exercise 2 - `modelf` above. If  you get the error kindly report it on this [Topic](https://discourse.deeplearning.ai/t/error-in-optional-ungraded-part-of-neural-machine-translation-w3a1/1096) on [Discourse](https://discourse.deeplearning.ai) as it'll help us improve our content.

If you haven’t joined our Discourse community you can do so by clicking on the link: http://bit.ly/dls-discourse

And don’t worry about the error, it will not affect the grading for this assignment.

In [ ]:
attention_map = plot_attention_map(model, human_vocab, inv_machine_vocab, "Tuesday 09 Oct 1993", num = 7, n_s = 64);

On the generated plot you can observe the values of the attention weights for each character of the predicted output. Examine this plot and check that the places where the network is paying attention makes sense to you.

In the date translation application, you will observe that most of the time attention helps predict the year, and doesn't have much impact on predicting the day or month.

### Congratulations!


You have come to the end of this assignment

#### Here's what you should remember

- Machine translation models can be used to map from one sequence to another. They are useful not just for translating human languages (like French->English) but also for tasks like date format translation.
- An attention mechanism allows a network to focus on the most relevant parts of the input when producing a specific part of the output.
- A network using an attention mechanism can translate from inputs of length $T_x$ to outputs of length $T_y$, where $T_x$ and $T_y$ can be different.
- You can visualize attention weights $\alpha^{\langle t,t' \rangle}$ to see what the network is paying attention to while generating each output.

Congratulations on finishing this assignment! You are now able to implement an attention model and use it to learn complex mappings from one sequence to another.